In [2]:
#Othello Game
import gymnasium as gym
import numpy as np
import time
import pygame
import sys
from gymnasium.spaces import MultiDiscrete, Discrete
from gymnasium import spaces

from pettingzoo import AECEnv
from pettingzoo.utils.agent_selector import agent_selector

import glob
import os
import time

from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker

import pettingzoo.utils
import random

In [3]:
#agents = ['player_W', 'player_B']
#_agent_selector = agent_selector(agents)
#agent_selection = _agent_selector.reset()

# Pygame

## User Interface

In [4]:
#Classes


#############################################################################################################################################

class OthelloDisplay():
    
    def __init__(self, display_width = 800, display_height = 600, fps=30):
        # Initialise the game.
        
        #Display the caption on top of the window.
        pygame.display.set_caption("Othello Game (an implementation of the popular game also known as Reversi)")

        #Set the window width and the window height.
        self.display_width = display_width
        self.display_height = display_height

        #Make the screen.
        self.Screen = pygame.display.set_mode((self.display_width, self.display_height))
        self.Background = pygame.Surface(self.Screen.get_size())
        self.Background.fill((255, 255, 255))
        self.Background = self.Background.convert()
        self.Screen.blit(self.Background, (0,0))

        #Set the Clock.
        self.Clock = pygame.time.Clock()  
        self.fps = fps

        #Define the colors
        self.black = (0,0,0)
        self.white = (255,255,255)
        self.cellgreen = (0,230,0)

In [5]:
class OthelloDisplay(OthelloDisplay):
    #Draw text on anew surface.
    def CreateText(self, text, Font):
        textSurface = Font.render(text, True, self.black)
        return textSurface, textSurface.get_rect()

In [6]:
class OthelloDisplay(OthelloDisplay):
    #To display the welcome message and the heading.
    def WelcomeMessageDisplay(self, text):
        Font = pygame.font.SysFont('freesansbold.tiff',50)
        TextSurf, TextRect = self.CreateText(text, Font)
        TextRect.center = (430,30)
        self.Background.blit(TextSurf, TextRect)

In [7]:
class OthelloDisplay(OthelloDisplay):
    #To display the choices the user has.
    def ChoiceDisplay(self, text, xcenter, ycenter):
        Font = pygame.font.SysFont('timesnewroman',30)
        TextSurf, TextRect = self.CreateText(text, Font)
        TextRect.center = (xcenter, ycenter)
        self.Background.blit(TextSurf, TextRect)

In [8]:
class OthelloDisplay(OthelloDisplay):
    #To dislay the disc choice buttons in the welcome screen.
    def DiscChoiceButtons(self):
        
        MousePos = pygame.mouse.get_pos()

        #Create white button
        WhiteButton = pygame.Surface((100,100))
        
        if 250 < MousePos[0] < 350 and 350 < MousePos[1] < 450:
            WhiteButton.fill((0,0,200))
        else:
            WhiteButton.fill((0,200,0))
        
        WhiteButton = WhiteButton.convert()
        self.Background.blit(WhiteButton, (250, 350))
        self.ChoiceDisplay('White', 300,400)


        #Create black button
        
        BlackButton = pygame.Surface((100,100))
        

        if 500 < MousePos[0] < 550 and 350 < MousePos[1] < 450:
            BlackButton.fill((0,0,200))
        else:
            BlackButton.fill((0,200,0))
        
        BlackButton = BlackButton.convert()
        self.Background.blit(BlackButton, (500, 350))
        self.ChoiceDisplay('Black', 550,400)

In [9]:
class OthelloDisplay(OthelloDisplay):
    #To activate the disc choice buttons in the welcome screen.
    def  ActivateButton(self, x, y):
        #Activate white button
    
        if 250 < x < 350 and 350 < y < 450:
            return 'W'
  
        #Activate Black Button

        if 500 < x < 550 and 350 < y < 450:
            return 'B'
        
        return ' '

In [10]:
class OthelloDisplay(OthelloDisplay):
    #The welcome screen loop.       
    def WelcomeScreen(self):
        MainLoop = True #The mainloop

        while MainLoop:

            self.Clock.tick(self.fps)
            self.WelcomeMessageDisplay('Welcome to Othello')
            self.ChoiceDisplay('Please choose the color of your disc', 430,200)
            self.DiscChoiceButtons()
            
            

            for Event in pygame.event.get():
                if Event.type == pygame.QUIT: #Player presses QUIT-button.
                    MainLoop = False
                    return 'Exit'
                elif Event.type == pygame.MOUSEBUTTONDOWN :
                    x, y = Event.pos
                    PlayerDisc = ' '
                    PlayerDisc = self.ActivateButton(x,y)
                    
                    if (PlayerDisc == 'W') or (PlayerDisc == 'B'):
                        MainLoop = False
                        return PlayerDisc
                    else:
                        MainLoop = True


            pygame.display.update()
            self.Screen.blit(self.Background, (0,0))
 

In [11]:
class OthelloDisplay(OthelloDisplay):
    #To create cells in the board.
    def CreateCell(self, radius, cellcolor, disccolor, x, y):

        MousePos = pygame.mouse.get_pos()
        
        Cell = pygame.Surface((2*radius,2*radius))
        
        if x < MousePos[0] < x + 2*radius and y < MousePos[1] < y + 2*radius:
            Cell.fill((0,0,200))
        else:
            Cell.fill(cellcolor)
            pygame.draw.rect(Cell, self.black, [0,0,2*radius, 2*radius], 3)
            pygame.draw.circle(Cell, disccolor, (radius, radius), radius-5)
        
        Cell = Cell.convert()
        self.Background.blit(Cell, ( x, y))

In [12]:
class OthelloDisplay(OthelloDisplay):
    #To create the game board.
    def CreateBoard(self, Board):

        for i in range(8):
            for j in range (8):
                if Board[i][j] == 'W':
                    self.CreateCell(20, self.cellgreen, self.white, x=15 + j*40, y=50 + i*40)
                elif Board[i][j] == 'B':
                    self.CreateCell(20, self.cellgreen, self.black, x=15 + j*40, y=50 + i*40)
                else:
                    self.CreateCell(20, self.cellgreen, self.cellgreen, x=15 + j*40, y=50 + i*40)

In [13]:
class OthelloDisplay(OthelloDisplay):
    #To display the game screen.
    def GameScreen(self, Board, Score):
        MainLoop = True #The mainloop

        while MainLoop:

            self.Clock.tick(self.fps)
            self.WelcomeMessageDisplay('Othello Game')
            self.ShowPoints(Score)
            self.DisplayMessage('Please make a move.')
            self.ExitButton()
            self.HintButton()
            self.CreateBoard(Board) 

            for Event in pygame.event.get():
                if Event.type == pygame.QUIT:
                    MainLoop = False
                    return 'Exit'
                elif Event.type == pygame.MOUSEBUTTONDOWN :
                    x, y = Event.pos

                    Move = [-1,-1]
                    self.CreateBoard(Board)
                    Move = self.ModifyBoard(x,y, Board)
                    i = Move[0]
                    j = Move[1]
                    if (i in range(8)) and (j in range(8)):
                        MainLoop = False
                        return Move
                    else:
                        MainLoop = True
                    
                    Exit = ' '
                    Exit = self.ActivateExitButton(x,y)
                    if Exit == 'Exit':
                        MainLoop = False
                        return 'Exit'
                    else:
                        MainLoop = True
                    
                    Hint = ' '
                    Hint = self.ActivateHintButton(x,y)
                    if Hint == 'Hint':
                        MainLoop = False
                        return 'Hint'
                    else:
                        MainLoop = True

            pygame.display.update()
            self.Screen.blit(self.Background, (0,0))

In [14]:
class OthelloDisplay(OthelloDisplay):
    #To show the current points.
    def ShowPoints(self, Score):
        Font = pygame.font.SysFont('mono',19)
        TextSurf, TextRect = self.CreateText("WhiteScore: " + str(Score['W']) + " BlackScore: " + str(Score['B']), Font)
        TextRect.center = (570,60)
        self.Background.blit(TextSurf, TextRect)

In [15]:
class OthelloDisplay(OthelloDisplay):
    #To display the desired message.
    def DisplayMessage(self, text):
        Font = pygame.font.SysFont('mono',19)
        TextSurf, TextRect = self.CreateText(text, Font)
        TextRect.center = (450,80)
        self.Background.blit(TextSurf, TextRect)

In [16]:
class OthelloDisplay(OthelloDisplay):
    #To display the exit button.
    def ExitButton(self):
        
        MousePos = pygame.mouse.get_pos()
        
        ExitButton = pygame.Surface((150,50))
        
        if 400 < MousePos[0] < 550 and 100 < MousePos[1] < 150:
            ExitButton.fill((0,0,200))
        else:
            ExitButton.fill((0,200,0))
        
        ExitButton = ExitButton.convert()
        self.Background.blit(ExitButton, (400, 100))
        self.ChoiceDisplay('Exit Game', 475,125)
        

In [17]:
class OthelloDisplay(OthelloDisplay):
    #To activate the exit button.
    def  ActivateExitButton(self, x, y):
        if 400 < x < 550 and 100 < y < 150:
            return 'Exit'
        

In [18]:
class OthelloDisplay(OthelloDisplay):
    #To display the hint button.
    def HintButton(self):
        
        MousePos = pygame.mouse.get_pos()
        
        HintButton = pygame.Surface((150,50))
        
        if 600 < MousePos[0] < 750 and 100 < MousePos[1] < 150:
            HintButton.fill((0,0,200))
        else:
            HintButton.fill((0,200,0))
        
        HintButton = HintButton.convert()
        self.Background.blit(HintButton, (600, 100))
        self.ChoiceDisplay('Get Hint', 675,125)
        

In [19]:
class OthelloDisplay(OthelloDisplay):
    #To activate the hint button.
    def  ActivateHintButton(self, x, y):
        if 600 < x < 750 and 100 < y < 150:
            return 'Hint'
        

In [20]:
class OthelloDisplay(OthelloDisplay):
    #To display the continue button.
    def ContinuePlayingButton(self):
        
        MousePos = pygame.mouse.get_pos()
        
        ContinuePlayingButton = pygame.Surface((250,50))
        
        if 500 < MousePos[0] < 750 and 200 < MousePos[1] < 250:
            ContinuePlayingButton.fill((0,0,200))
        else:
            ContinuePlayingButton.fill((0,200,0))
        
        ContinuePlayingButton = ContinuePlayingButton.convert()
        self.Background.blit(ContinuePlayingButton, (500, 200))
        self.ChoiceDisplay('Continue Playing', 630,225)
        

In [21]:
class OthelloDisplay(OthelloDisplay):
    #To activate the continue button.
    def  ActivateContinuePlayingButton(self, x, y):
        if 500 < x < 750 and 200 < y < 250:
            return 'Continue Playing'    

In [22]:
class OthelloDisplay(OthelloDisplay):
    #To activate the board.
    def ModifyBoard(self,cx,cy, Board):
        radius = 20
        cellcolor = self.cellgreen
        disccolor = self.black
        for i in range(8):
            for j in range (8):
                Cell = pygame.Surface((2*radius,2*radius))
                Cell.fill(cellcolor)
                pygame.draw.rect(Cell, self.black, [0,0,2*radius, 2*radius], 3)
                pygame.draw.circle(Cell, disccolor, (radius, radius), radius-5)
                x=15 + j*40
                y=50 + i*40

                if x < cx < x + 2*radius and y < cy < y + 2*radius:

                    if Board[i][j] == ' ':
                        Cell = Cell.convert()
                        self.Background.blit(Cell, ( x, y))
                        return [i,j]
                        
        return [-1,-1]
    

In [23]:
class OthelloDisplay(OthelloDisplay):                    
    #To display the intermediate screen for the hint.
    def IntermediateScreen(self, TemporaryBoard, LegalMoves, Score):
        MainLoop = True #The mainloop

        while MainLoop:

            self.Clock.tick(self.fps)
            self.WelcomeMessageDisplay('Othello Game')
            self.ShowPoints(Score)
            self.DisplayMessage('Please make a move.')
            self.ContinuePlayingButton()
            self.CreateHintBoard(TemporaryBoard, LegalMoves)

            for Event in pygame.event.get():
                if Event.type == pygame.QUIT:
                    MainLoop = False
                    return 'Exit'
                elif Event.type == pygame.MOUSEBUTTONDOWN :
                    x, y = Event.pos
    
                    ContinuePlaying = ' '
                    ContinuePlaying = self.ActivateContinuePlayingButton(x,y)
                    if ContinuePlaying == 'Continue Playing':
                        MainLoop = False
                        return ContinuePlaying
                    else:
                        MainLoop = True
                    

            pygame.display.update()
            self.Screen.blit(self.Background, (0,0))
            

In [24]:
class OthelloDisplay(OthelloDisplay):        
    #To create the hint board.
    def CreateHintBoard(self,TemporaryBoard, LegalMoves):

        for i in range(8):
            for j in range (8):
                if TemporaryBoard[i][j] == 'W':
                    self.CreateCell(20, self.cellgreen, self.white, x=15 + j*40, y=50 + i*40)
                elif TemporaryBoard[i][j] == 'B':
                    self.CreateCell(20, self.cellgreen, self.black, x=15 + j*40, y=50 + i*40)
                elif [i,j] in LegalMoves:
                    self.CreateCell(20, self.cellgreen, (255,255,0), x=15 + j*40, y=50 + i*40)
                else:
                    self.CreateCell(20, self.cellgreen, self.cellgreen, x=15 + j*40, y=50 + i*40)

In [25]:
class OthelloDisplay(OthelloDisplay):
    #To display the illegal move screen
    def IllegalMoveScreen(self, TemporaryBoard, Score):
        MainLoop = True #The mainloop

        while MainLoop:

            self.Clock.tick(self.fps)
            self.WelcomeMessageDisplay('Othello Game')
            self.ShowPoints(Score)
            self.DisplayMessage('Illegal move.')
            self.ContinuePlayingButton()
            self.CreateBoard(TemporaryBoard)

            for Event in pygame.event.get():
                if Event.type == pygame.QUIT:
                    MainLoop = False
                    return 'Exit'
                elif Event.type == pygame.MOUSEBUTTONDOWN :
                    x, y = Event.pos
    
                    ContinuePlaying = ' '
                    ContinuePlaying = self.ActivateContinuePlayingButton(x,y)
                    if ContinuePlaying == 'Continue Playing':
                        MainLoop = False
                        return ContinuePlaying
                    else:
                        MainLoop = True
                    

            pygame.display.update()
            self.Screen.blit(self.Background, (0,0))

In [26]:
class OthelloDisplay(OthelloDisplay):
    #To display the final screen.
    def FinalScreen(self, TemporaryBoard, Score):
        MainLoop = True #The mainloop

        while MainLoop:

            self.Clock.tick(self.fps)
            self.WelcomeMessageDisplay('Othello Game')
            self.ShowPoints(Score)
            if Score['W'] > Score['B']:
                self.DisplayMessage('White Wins')
            elif Score['W'] < Score['B']:
                self.DisplayMessage('Black Wins')
            else:
                self.DisplayMessage('Game has Tied')
            self.ExitButton()
            self.CreateBoard(TemporaryBoard)

            for Event in pygame.event.get():
                if Event.type == pygame.QUIT:
                    MainLoop = False
                    return 'Exit'
                elif Event.type == pygame.MOUSEBUTTONDOWN :
                    x, y = Event.pos

                    Exit = ' '
                    Exit = self.ActivateExitButton(x,y)
                    if Exit == 'Exit':
                        MainLoop = False
                        return Exit
                    else:
                        MainLoop = True
                    
            pygame.display.update()
            self.Screen.blit(self.Background, (0,0))

In [27]:
class OthelloDisplay(OthelloDisplay):            
    #To display the screen after the computer made a move.
    def SeeComputerMoveScreen(self, TemporaryBoard, Score):
        MainLoop = True #The mainloop

        while MainLoop:

            self.Clock.tick(self.fps)
            self.WelcomeMessageDisplay('Othello Game')
            self.ShowPoints(Score)
            self.DisplayMessage("Your Move is:")
            self.ContinuePlayingButton()
            self.CreateBoard(TemporaryBoard)

            for Event in pygame.event.get():
                if Event.type == pygame.QUIT:
                    MainLoop = False
                    return 'Exit'
                elif Event.type == pygame.MOUSEBUTTONDOWN :
                    x, y = Event.pos

                    ContinuePlaying = ' '
                    ContinuePlaying = self.ActivateContinuePlayingButton(x,y)
                    if ContinuePlaying == 'Continue Playing':
                        MainLoop = False
                        return ContinuePlaying
                    else:
                        MainLoop = True
                    
            pygame.display.update()
            self.Screen.blit(self.Background, (0,0))

## Othello Board

In [30]:
          
#############################################################################################################################################

class OthelloBoardArray():
   
    def __init__ (self):

        '''
        Board is like:
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']                                
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
        '''

        self.Board = [[' '] * 8 for i in range(8)]
        self.Board[3][3], self.Board[4][4] = 'W', 'W'
        self.Board[3][4], self.Board[4][3] = 'B', 'B'

In [31]:
class OthelloBoardArray(OthelloBoardArray):        
    #Creatin a temporary board.
    def MakeTempBoard(self, GivenBoard):
        TempBoard = [[' '] * 8 for i in range(8)]
        for i in range(8):
            for j in range(8):
                TempBoard[i][j] = GivenBoard[i][j]

        return TempBoard
      

In [32]:
class OthelloBoardArray(OthelloBoardArray):             
    #Check if the move is legal and return the cells to be filled if it so.
    def isLegalMove(self, Disc, CellLocation, GivenBoard):
        
        '''
        x_loc = CellLocation[0]
        y_loc = CellLocation[1]
        
        if GivenBoard[x_loc][y_loc] == 'W' or GivenBoard[x_loc][y_loc] == 'B':
            print(x_loc, y_loc)
            return 'False'
        '''   
        
        # Setting the discs
        if Disc == 'W':
            OpponentDisc = 'B'
        else:
            OpponentDisc = 'W'    

        # Creating the board directions
        EightDirections = []
        
        for i in range(-1,2):
            for j in range(-1,2):
                EightDirections.append([i,j])

        EightDirections.remove([0,0])

        #Creating an array to store the locations of cells to be flipped.
        CellsToBeFlipped = []

        #Checking all the Directions
        for Direction in EightDirections:
            
            XDirection = Direction[0]
            YDirection = Direction[1]
            x = CellLocation[0]
            y = CellLocation[1]

            x = x + XDirection
            y = y + YDirection

            TempCellStorage = []

            while(((x in range(8)) and (y in range(8))) and GivenBoard[x][y]==OpponentDisc):
                TempCellStorage.append([x,y])

                x = x + XDirection
                y = y + YDirection

                if not((x in range(8)) and (y in range(8))) or (GivenBoard[x][y]==' '):
                    TempCellStorage.clear()
                    break

                if (GivenBoard[x][y]==Disc):
                    break


            CellsToBeFlipped.extend(TempCellStorage)

        if len(CellsToBeFlipped) == 0:
            return 'False'
        else:
            return CellsToBeFlipped

In [33]:
class OthelloBoardArray(OthelloBoardArray): 
    #Find all possible legal moves.
    def FindAllLegalMoves(self, Disc, GivenBoard):
        
        LegalMoves = []
        
        for i in range(8):
            for j in range(8):
                if (GivenBoard[i][j] == ' ') and (self.isLegalMove(Disc, [i,j], GivenBoard) != 'False'):
                    LegalMoves.append([i,j])

        return LegalMoves

In [34]:
class OthelloBoardArray(OthelloBoardArray): 
    #Calculate the score.
    def CalculateScore(self, GivenBoard):

        WhiteScore, BlackScore = 0,0

        for i in range(8):
            for j in range(8):
                if (GivenBoard[i][j]=='W'):
                    WhiteScore += 1
                if(GivenBoard[i][j]=='B'):
                    BlackScore += 1

        return {'W': WhiteScore,'B': BlackScore}

In [35]:
class OthelloBoardArray(OthelloBoardArray): 
    #Update the Board.
    def UpdateBoard(self, GivenBoard, CellsToBeFlipped):

        for Cell in CellsToBeFlipped:
            #print("Cell is: ", Cell)
            x = Cell[0]
            y = Cell[1]
            if GivenBoard[x][y] == 'W':
                GivenBoard[x][y] = 'B'
            else:
                GivenBoard[x][y] = 'W'

        return  GivenBoard

In [36]:
class OthelloBoardArray(OthelloBoardArray): 
    #Check if the move is a corner move.
    def isCornerMove(self, x, y):
        return (x == 0 or x == 7) and (y == 0 or y == 7)

In [37]:
class OthelloBoardArray(OthelloBoardArray): 
    #Choose the move for the computer.
    def ChooseComputerMove(self, Disc, GivenBoard):
        # Setting the discs
        if Disc == 'W':
            OpponentDisc = 'B'
        else:
            OpponentDisc = 'W'

        LegalMoves = self.FindAllLegalMoves(OpponentDisc, GivenBoard)
        
        #print("LegalMoves: ", LegalMoves)

        MaxScore = 0
        MaxMove = [-1,-1]

        for Move in LegalMoves:
            x = Move[0]
            y = Move[1]

            if self.isCornerMove(x, y):
                MaxMove = [x,y]
                break

            TempBoard = self.MakeTempBoard(GivenBoard)
            TempBoard[x][y] = OpponentDisc
            CellsToBeFlipped = self.isLegalMove(OpponentDisc, Move, TempBoard)
            #print("CellsToBeFlipped: ", CellsToBeFlipped)
            #return [-1,-1]
            if CellsToBeFlipped != 'False':
                TempBoard = self.UpdateBoard(TempBoard, CellsToBeFlipped)
                Score = self.CalculateScore(TempBoard)
            
                if Score[OpponentDisc] > MaxScore:
                    MaxScore = Score[OpponentDisc]
                    MaxMove = Move

        return MaxMove

In [38]:
#############################################################################################################################################
'''
pygame.init()
OpeningScreen = OthelloDisplay()
PlayerDisc = OpeningScreen.WelcomeScreen()
if PlayerDisc == 'W':
    ComputerDisc = 'B'
else:
    ComputerDisc = 'W'

Turn = 'Player'

OthelloGame = OthelloBoardArray()
OthelloBoard = OthelloGame.Board


while True:
    if PlayerDisc == 'Exit':
        pygame.quit()
        break
    
    if Turn == 'Player':
        GameDisplay = OthelloDisplay()
        Score = OthelloGame.CalculateScore(OthelloBoard)
        Move = GameDisplay.GameScreen(OthelloBoard, Score) # update gamescreen(self, score)

        if Move == 'Exit':
            #print("Exit")
            pygame.quit()
            break
        elif Move == 'Hint':
            TemporaryBoard = OthelloBoard
            LegalMoves = OthelloGame.FindAllLegalMoves(PlayerDisc, TemporaryBoard)
            IntermediateDisplay = OthelloDisplay()
            Move = IntermediateDisplay.IntermediateScreen(TemporaryBoard, LegalMoves, Score)
            if Move == 'Continue Playing':
                #print ("Hello")
                continue
            if Move == 'Exit':
                pygame.quit()
                break
        else:
            CellsToBeFlipped = OthelloGame.isLegalMove(PlayerDisc, Move, OthelloBoard)
            if (CellsToBeFlipped == 'False'):
                IllegalMoveDisplay = OthelloDisplay()
                Move = IllegalMoveDisplay.IllegalMoveScreen(OthelloBoard, Score)
                if Move == 'Continue Playing':
                    #print ("Hello")
                    continue
                if Move == 'Exit':
                    pygame.quit()
                    break

            else:
                x = Move[0]
                y = Move[1]
                OthelloBoard[x][y] = PlayerDisc
                OthelloBoard = OthelloGame.UpdateBoard(OthelloBoard, CellsToBeFlipped)
                Score = OthelloGame.CalculateScore(OthelloBoard)
                
                LegalMoves = OthelloGame.FindAllLegalMoves(ComputerDisc, OthelloBoard)

                if len(LegalMoves) == 0:
                    FinalDisplay = OthelloDisplay()
                    Move = FinalDisplay.FinalScreen(OthelloBoard, Score)
                    if Move == 'Exit':
                        pygame.quit()
                        break
                else:
                    ToSeeComputerMove = OthelloDisplay()
                    Move = ToSeeComputerMove.SeeComputerMoveScreen(OthelloBoard, Score)
                    if Move == 'Continue Playing':
                        #print ("Hello")
                        Turn = 'Computer'
                        continue
                    if Move == 'Exit':
                        pygame.quit()
                        break

    else:
        ComputerMove = OthelloGame.ChooseComputerMove(PlayerDisc, OthelloBoard)
        #break
        x = ComputerMove[0]
        y = ComputerMove[1]
        OthelloBoard[x][y] = ComputerDisc
        CellsToBeFlipped = OthelloGame.isLegalMove(ComputerDisc, ComputerMove, OthelloBoard)
        #print(CellsToBeFlipped)
        OthelloBoard = OthelloGame.UpdateBoard(OthelloBoard, CellsToBeFlipped)

        LegalMoves = OthelloGame.FindAllLegalMoves(PlayerDisc, OthelloBoard)

        if len(LegalMoves) == 0:
            FinalDisplay = OthelloDisplay()
            Move = FinalDisplay.FinalScreen(OthelloBoard, Score)
            if Move == 'Exit':
                pygame.quit()
                break
        else:
            Turn = 'Player'
        

pygame.quit()
'''

'\npygame.init()\nOpeningScreen = OthelloDisplay()\nPlayerDisc = OpeningScreen.WelcomeScreen()\nif PlayerDisc == \'W\':\n    ComputerDisc = \'B\'\nelse:\n    ComputerDisc = \'W\'\n\nTurn = \'Player\'\n\nOthelloGame = OthelloBoardArray()\nOthelloBoard = OthelloGame.Board\n\n\nwhile True:\n    if PlayerDisc == \'Exit\':\n        pygame.quit()\n        break\n    \n    if Turn == \'Player\':\n        GameDisplay = OthelloDisplay()\n        Score = OthelloGame.CalculateScore(OthelloBoard)\n        Move = GameDisplay.GameScreen(OthelloBoard, Score) # update gamescreen(self, score)\n\n        if Move == \'Exit\':\n            #print("Exit")\n            pygame.quit()\n            break\n        elif Move == \'Hint\':\n            TemporaryBoard = OthelloBoard\n            LegalMoves = OthelloGame.FindAllLegalMoves(PlayerDisc, TemporaryBoard)\n            IntermediateDisplay = OthelloDisplay()\n            Move = IntermediateDisplay.IntermediateScreen(TemporaryBoard, LegalMoves, Score)\n     

In [39]:
# Make it illegal to click on an occupied spot.

# Gym

In [40]:
class OthelloEnv(AECEnv):
    metadata = {'name': 'othello', 'render_modes': ['human_player', 'matrix'], 'render_fps': 3}
    
    def __init__(self, render_mode='matrix'):
        # Initialize the parent class.
        super().__init__()
        
        # Render mode fiddling
        self.render_mode = render_mode
        assert self.render_mode is None or self.render_mode in self.metadata["render_modes"]
        if self.render_mode == 'human_player':
            pass
            return

        # Set the agents.
        self.agents = ["player_W", "player_B"]
        self.possible_agents = ["player_W", "player_B"]
        self.agent_to_num = {'player_W': 0, 'player_B': 1}

        # reset
        self.reset()

        # action spaces
        self.action_spaces = {agent: Discrete(64) for agent in self.agents}

        # observation spaces
        self.observation_spaces = { 
            agent: spaces.Dict(
                {
                    "observation":MultiDiscrete(np.array([3 for i in range(64)]), dtype=np.int8),
                    "action_mask":MultiDiscrete(np.array([2 for i in range(64)]), dtype=np.int8),
                }
            )
            for agent in self.agents
        }

    def observe(self, agent):
        Disc = agent[-1]
        LegalMoves = self.OthelloGame.FindAllLegalMoves(Disc, self.OthelloBoard)

        action_mask = np.zeros(64, dtype=np.int8)

        for Move in LegalMoves:
            action = self.cell_to_action(Move)
            action_mask[action] = 1

        return {"observation": self.board_to_obs(self.OthelloBoard) , "action_mask": action_mask}

    def reset(self, seed=None, options=None):
        #Set random seed
        random.seed(seed)
        
        self.OthelloGame = OthelloBoardArray()
        self.OthelloBoard = self.OthelloGame.Board

        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}

        self.agent_selection = random.choice(self.agents)
        
        return

    def select_next(self, agent):
        if agent == 'player_W':
            return 'player_B'
        else:
            return 'player_W'

    def step(self, action):
        Move = self.action_to_cell(action)
        
        # Check if the move is legal.
        self.PlayerDisc = self.agent_selection[-1]
        if self.PlayerDisc == 'W':
            self.OpponentDisc = 'B'
        else:
            self.OpponentDisc = 'W'
                
        CellsToBeFlipped = self.OthelloGame.isLegalMove(self.PlayerDisc, Move, self.OthelloBoard)
        if (CellsToBeFlipped == 'False'):
            #print("Illegal Move")
            '''
            self.terminations = {agent: True for agent in self.agents}
            # Calculate the score
            Score = self.OthelloGame.CalculateScore(self.OthelloBoard)
            # Calculate the reward. Small reward for intermediate scores, big rewards for winning the game.
    
            for agent in self.agents:
                Disc = agent[-1]
                if Disc == 'W':
                    OppDisc = 'B'
                else:
                    OppDisc = 'W'
    
                reward = Score[Disc] - Score[OppDisc]
                self.rewards[agent] = reward
            self.rewards[self.agent_selection] = self.rewards[self.agent_selection] - 100
            winner = self.select_next(self.agent_selection)
            self.rewards[winner] = self.rewards[winner] + 100
            for agent in self.agents:
                self._cumulative_rewards[agent] = self.rewards[agent]
            self.agent_selection = self.select_next(self.agent_selection)
            self.close()
            '''
            self._was_dead_step(None)
            return
        
        # Update the board
        x = Move[0]
        y = Move[1]
        self.OthelloBoard[x][y] = self.PlayerDisc
        self.OthelloBoard = self.OthelloGame.UpdateBoard(self.OthelloBoard, CellsToBeFlipped)
        
        # Calculate the score
        Score = self.OthelloGame.CalculateScore(self.OthelloBoard)
        # Calculate the reward. Small reward for intermediate scores, big rewards for winning the game.

        for agent in self.agents:
            Disc = agent[-1]
            if Disc == 'W':
                OppDisc = 'B'
            else:
                OppDisc = 'W'

            reward = Score[Disc] - Score[OppDisc]
            self.rewards[agent] = reward
        
        # Check if game has ended.
        LegalMoves = self.OthelloGame.FindAllLegalMoves(self.OpponentDisc, self.OthelloBoard)
        
        if len(LegalMoves) == 0:
            self.render()
            if (Score['W'] - Score['B']) == 0:
                self.rewards['player_W'] = self.rewards['player_W'] - 100
                self.rewards['player_B'] = self.rewards['player_B'] - 100
            elif (Score['W'] - Score['B']) > 0:
                self.rewards['player_W'] = self.rewards['player_W'] + 100
                self.rewards['player_B'] = self.rewards['player_B'] - 100
            else:
                self.rewards['player_W'] = self.rewards['player_W'] - 100
                self.rewards['player_B'] = self.rewards['player_B'] + 100
                
            self.terminations = {agent: True for agent in self.agents}
            for agent in self.agents:
                self._cumulative_rewards[agent] = self.rewards[agent]
                #self._cumulative_rewards[agent] = self._cumulative_rewards[agent] + self.rewards[agent]

            self.agent_selection = self.select_next(self.agent_selection)
            self.close()
            return self.board_to_obs(self.OthelloBoard), self.rewards, self.terminations, self.truncations, self.infos
        else:
            self.render()
            for agent in self.agents:
                self._cumulative_rewards[agent] = self.rewards[agent]
                #self._cumulative_rewards[agent] = self._cumulative_rewards[agent] + self.rewards[agent]

            self.agent_selection = self.select_next(self.agent_selection)
            return self.board_to_obs(self.OthelloBoard), self.rewards, self.terminations, self.truncations, self.infos
        
        # Render the board to display the current status.
        # Pause the screen to allow the user to comprehend the screen.
        # time.sleep(5)
        # Render the screen
                
        return self.board_to_obs(self.OthelloBoard), self.rewards, self.terminations, self.truncations, self.infos
       
    def render(self):
        '''
        print("###################################################")
        print("The current board is: ")
        for row in self.OthelloBoard:
            print(row)
        print("###################################################")
        '''
        pass
        return

    def observation_space(self, agent):
        # Observation Space
        # 0 - empty cell
        # 1 - white disc
        # 2 - black disc
        return self.observation_spaces[agent]

    def action_space(self, agent):
        # Action Space
        #[0,1,2,3,4,5,6,7]
        #[ ... ]
        #[56,57,58,59,60,61,62,63]
        #col = n%8
        #row = int(n/8)
        return self.action_spaces[agent]
 
    def obs_to_board(self, obs_arr):
        board = [[0 for i in range(8)] for j in range(8)]  
        obs_to_cell_dict = { 0:' ', 1:'W', 2:'B' }
        for i in range(64):
            row = int(i/8)
            col = i%8
            board[row][col] = obs_to_cell_dict[obs_arr[i]]

        return board

    def board_to_obs(self, board):
        cell_to_obs_dict = {' ':0, 'W':1, 'B':2 }
        obs_arr = []
        for row in board:
            obs_arr.extend(row)
        obs_arr = [cell_to_obs_dict[i] for i in obs_arr]
        obs_arr = np.array(obs_arr)
        
        return obs_arr

    def action_to_cell(self, action):
        return [int(action/8), action%8]

    def cell_to_action(self, cell):
        row, col = cell
        action = row*8 + col
        
        return action
         
    def close(self):
        pass
        #print("The game is over.")
        return
         
    def human_player(self):
        pygame.init()
        OpeningScreen = OthelloDisplay()
        PlayerDisc = OpeningScreen.WelcomeScreen()
        if PlayerDisc == 'W':
            ComputerDisc = 'B'
        else:
            ComputerDisc = 'W'

        Turn = 'Player'

        OthelloGame = OthelloBoardArray(PlayerDisc)
        OthelloBoard = OthelloGame.Board


        while True:
            if PlayerDisc == 'Exit':
                pygame.quit()
                break

            if Turn == 'Player':
                GameDisplay = OthelloDisplay()
                Score = OthelloGame.CalculateScore(OthelloBoard)
                Move = GameDisplay.GameScreen(OthelloBoard, Score) # update gamescreen(self, score)

                if Move == 'Exit':
                    #print("Exit")
                    pygame.quit()
                    break
                elif Move == 'Hint':
                    TemporaryBoard = OthelloBoard
                    LegalMoves = OthelloGame.FindAllLegalMoves(PlayerDisc, TemporaryBoard)
                    IntermediateDisplay = OthelloDisplay()
                    Move = IntermediateDisplay.IntermediateScreen(TemporaryBoard, LegalMoves, Score)
                    if Move == 'Continue Playing':
                        #print ("Hello")
                        continue
                    if Move == 'Exit':
                        pygame.quit()
                        break
                else:
                    CellsToBeFlipped = OthelloGame.isLegalMove(PlayerDisc, Move, OthelloBoard)
                    if (CellsToBeFlipped == 'False'):
                        IllegalMoveDisplay = OthelloDisplay()
                        Move = IllegalMoveDisplay.IllegalMoveScreen(OthelloBoard, Score)
                        if Move == 'Continue Playing':
                            #print ("Hello")
                            continue
                        if Move == 'Exit':
                            pygame.quit()
                            break

                    else:
                        x = Move[0]
                        y = Move[1]
                        OthelloBoard[x][y] = PlayerDisc
                        OthelloBoard = OthelloGame.UpdateBoard(OthelloBoard, CellsToBeFlipped)
                        Score = OthelloGame.CalculateScore(OthelloBoard)

                        LegalMoves = OthelloGame.FindAllLegalMoves(ComputerDisc, OthelloBoard)

                        if len(LegalMoves) == 0:
                            FinalDisplay = OthelloDisplay()
                            Move = FinalDisplay.FinalScreen(OthelloBoard, Score)
                            if Move == 'Exit':
                                pygame.quit()
                                break
                        else:
                            ToSeeComputerMove = OthelloDisplay()
                            Move = ToSeeComputerMove.SeeComputerMoveScreen(OthelloBoard, Score)
                            if Move == 'Continue Playing':
                                #print ("Hello")
                                Turn = 'Computer'
                                continue
                            if Move == 'Exit':
                                pygame.quit()
                                break

            else:
                ComputerMove = OthelloGame.ChooseComputerMove(PlayerDisc, OthelloBoard)
                #break
                x = ComputerMove[0]
                y = ComputerMove[1]
                OthelloBoard[x][y] = ComputerDisc
                CellsToBeFlipped = OthelloGame.isLegalMove(ComputerDisc, ComputerMove, OthelloBoard)
                #print(CellsToBeFlipped)
                OthelloBoard = OthelloGame.UpdateBoard(OthelloBoard, CellsToBeFlipped)

                LegalMoves = OthelloGame.FindAllLegalMoves(PlayerDisc, OthelloBoard)

                if len(LegalMoves) == 0:
                    FinalDisplay = OthelloDisplay()
                    Move = FinalDisplay.FinalScreen(OthelloBoard, Score)
                    if Move == 'Exit':
                        pygame.quit()
                        break
                else:
                    Turn = 'Player'


        pygame.quit()
        return